In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import pandas_to_fits, desi_reliable_magnitudes
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import seaborn as sns
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Counterparts and field sources catalogs construction

## Load DESI-CSC 30 arcsec cone search match

In [3]:
desi_csc_orig = pd.read_pickle(data_path+'/csc_allsky_desi_r30_gaia_dered.gz_pkl',compression='gzip')

In [4]:
desi_csc_orig = desi_reliable_magnitudes(desi_csc_orig) #make cuts for SNR of flux + add colors
desi_csc_orig

/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
0,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9135,336127,359.993680,0.721193,1,0.264035,0.761373,1.464151,-2.279967,-0.936121,-109.650260,-1274.12740,2924.99000,2268.87720,293.554100,3.076859,0.628511,0.000788,0.000010,23.857758,22.736668,22.052845,NaN,NaN,NaN,NaN,14.279862,36.266293,25.085913,-3.999284,-0.742145,-3.078044,-4.096002,0.0,0.0,0.0,0.0,0.0,0.0,27.164009,9010_336127_9135,23.945846,23.945846,23.857758,22.796006,22.796006,22.736668,22.086035,22.086035,22.052845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.520281e-15,1.121090,1.804913,0.683823,NaN,NaN,NaN,NaN,NaN,NaN
1,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9169,336127,359.994202,0.722597,1,0.994499,2.786103,5.748118,6.166792,6.704570,-0.933517,1110.99740,2478.61230,1837.17550,249.912500,2.884505,0.622227,0.000782,0.000011,22.417845,21.328133,20.567974,20.519806,20.430973,NaN,14.885468,49.511795,119.418724,90.869820,10.473576,5.288656,-0.026110,3.618951,0.0,0.0,0.0,0.0,0.0,0.0,23.070881,9010_336127_9169,22.505989,22.505989,22.417845,21.387507,21.387507,21.328133,20.601186,20.601186,20.567974,20.524852,20.524852,20.519806,17.825852,20.434073,20.434073,20.430973,17.095073,NaN,NaN,NaN,NaN,14.885717,NaN,NaN,NaN,9.894579e-15,1.089712,1.849871,0.760159,0.048168,0.897160,0.088833,NaN,NaN,NaN
2,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9295,336127,359.997367,0.721166,1,0.287270,0.334849,0.528142,-1.608118,0.423848,16.828413,692.35394,2346.16870,1762.25840,213.479600,2.978809,0.614420,0.000778,0.000010,23.765974,23.628395,23.159851,NaN,23.428870,19.434230,15.398930,13.914554,14.056722,7.716652,-2.775487,0.332233,0.469412,2.225948,0.0,0.0,0.0,0.0,0.0,0.0,17.638166,9010_336127_9295,23.854274,23.854274,23.765974,23.687877,23.687877,23.628395,23.193123,23.193123,23.159851,NaN,NaN,NaN,NaN,23.431975,NaN,NaN,NaN,19.434892,NaN,NaN,NaN,15.399180,NaN,NaN,NaN,9.091730e-16,0.137579,0.606123,0.468544,NaN,NaN,NaN,NaN,NaN,NaN
3,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9271,336127,359.996619,0.722544,1,0.195305,0.580013,1.060394,-1.374197,0.465793,55.299458,840.05010,1941.29630,1459.10840,181.554780,2.827259,0.595737,0.000749,0.000010,24.184935,23.031937,22.403067,NaN,23.326414,18.142536,15.188987,8.605151,22.155508,14.287992,-2.310637,0.359518,1.513426,2.704489,0.0,0.0,0.0,0.0,0.0,0.0,15.723989,9010_336127_9271,24.273217,24.273217,24.184935,23.091406,23.091406,23.031937,22.436332,22.436332,22.403067,NaN,NaN,NaN,NaN,23.329518,NaN,NaN,NaN,18.143198,NaN,NaN,NaN,15.189237,NaN,NaN,NaN,1.825409e-15,1.152998,1.781868,0.628870,NaN,NaN,NaN,NaN,NaN,NaN
4,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9360,336127,359.998741,0.722629,1,0.162211,0.722331,4.141845,10.932156,10.346854,281.147600,-6404.08400,1451.76900,1055.47450,125.596010,2.283064,0.530093,0.000696,0.000010,24.386390,22.793606,20.923704,19.898174,19.959871,16.377000,NaN,6.180582,23.467144,46.417503,16.518280,7.533284,7.418694,-20.397125,0.0,0.0,0.0,

### Primary filters and DESI duplicates (ra,dec) removal

In [5]:
#step 1: filter brick ID
#next sort by name_csc, ra, dec and flux_g, with flux_g from largest to smallest
desi_csc = desi_csc_orig.query('brick_primary == True')
print('desi reliable catalog:', len(desi_csc))
desi_csc = desi_csc.sort_values(by=['csc_name', 'csc_ra', 'csc_dec', 'flux_g'], ascending=[True, True, True, False])

#step 2: remove duplicates in ['ra_csc', 'dec_csc', 'ra', 'dec'] and keep the one with the highest flux_g 
desi_csc = desi_csc.drop_duplicates(subset=['csc_ra', 'csc_dec', 'ra', 'dec'], keep='first')

print('desi reliable catalog after non-primary brick and duplicates removal:', len(desi_csc))


#drop LH area
ra_min = 154.763934
ra_max = 167.084550
dec_min = 54.124219
dec_max = 61.236185
in_lh_mask = desi_csc.eval('(csc_ra >= @ra_min) & (csc_ra <= @ra_max) & (csc_dec >= @dec_min) & (csc_dec <= @dec_max)')
desi_csc = desi_csc[~in_lh_mask]

#sort by sep_csc so that the closest objects are first
desi_csc.sort_values(by=['csc_name', 'dist_arcsec'], inplace=True)
desi_csc.sample(10)

desi reliable catalog: 3249397
desi reliable catalog after non-primary brick and duplicates removal: 3249392


,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
1312586,2CXO J085438.0+502750,133.658659,50.464050,NaN,0.985952,2.757854,9011,2351,586083,133.648826,50.468483,1,0.066901,0.336241,7.118179,6.037254,6.934602,28.685610,39.83331,476.06912,203.845110,100.111980,3.639785,0.858982,0.001180,0.000015,25.364000,23.634590,20.341791,20.543755,20.394900,18.855297,18.499180,1.459713,4.800664,71.221634,11.518007,6.427078,0.985295,0.152666,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,27.610506,9011_586083_2351,25.436418,NaN,NaN,23.683373,23.683373,23.634590,20.369078,20.369078,20.341791,20.547901,20.547901,20.543755,17.848901,20.397446,20.397446,20.394900,17.058446,18.855840,NaN,NaN,NaN,18.499384,NaN,NaN,NaN,1.218626e-14,NaN,NaN,3.292799,-0.201964,3.239690,0.148855,NaN,NaN,NaN
2223305,2CXO J133205.4+060547,203.022863,6.096389,3.122904e-14,0.452016,1.264355,9010,3272,365698,203.016827,6.095762,1,0.655324,0.479718,0.345021,1.532665,0.826241,43.082287,-865.26605,179.53910,80.648240,9.050529,2.866769,0.616138,0.001314,0.000019,22.864115,23.233713,23.619686,22.030958,22.703901,18.413542,NaN,8.780836,4.308075,1.037966,2.595038,0.648554,1.561925,-3.733164,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,21.724974,9010_365698_3272,22.958860,22.958860,22.864115,23.297535,23.297535,23.233713,23.655386,NaN,NaN,22.036382,NaN,NaN,NaN,22.707233,NaN,NaN,NaN,18.414253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.369598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66583,2CXO J002233.6+001208,5.640124,0.202439,5.218490e-16,1.593101,4.456142,9010,5081,331830,5.638582,0.196294,1,0.159393,0.228243,0.295384,1.461727,1.602158,70.684900,860.34780,2422.42460,1505.663200,238.887480,2.987206,0.601675,0.000608,0.000008,24.408928,24.046820,23.792046,22.082973,21.985252,17.876047,15.163074,7.845047,8.856473,4.565444,2.526381,1.242757,1.742766,2.485929,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,22.807182,9010_331830_5081,24.493827,24.493827,24.408928,24.104006,24.104006,24.046820,23.824033,23.824033,23.792046,22.087834,NaN,NaN,NaN,21.988237,NaN,NaN,NaN,17.876683,NaN,NaN,NaN,15.163315,NaN,NaN,NaN,5.078887e-16,0.362108,0.616882,0.254774,NaN,NaN,NaN,NaN,NaN,NaN
3056333,2CXO J224029.3+032331,340.122293,3.391974,0.000000e+00,0.439695,1.229892,9010,2744,351884,340.120401,3.390185,1,0.433981,0.141673,0.495041,0.324940,-1.596317,-28.834700,-449.67407,380.18420,74.373420,24.886358,2.585505,0.553985,0.000376,0.000006,23.180254,24.469501,23.178217,23.707552,NaN,NaN,NaN,8.461905,1.221785,2.469572,0.522487,-1.188141,-0.558786,-1.127223,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,9.363601,9010_351884_2744,23.406323,23.406323,23.180254,24.621782,NaN,NaN,23.263397,NaN,NaN,23.720492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117450,2CXO J061547.3-574939,93.947377,-57.827719,3.165654e-14,0.415634,1.162592,9010,6022,51229,93.939644,-57.827763,1,4.082197,16.506647,62.256620,40.084938,25.843357,22.061050,-1196.72110,3212.35180,1515.208600,228.236540,4.172762,1.673085,0.004136,0.000047,20.838097,19.365137,17.963795,18.484837,18.964394,19.139925,NaN,231.369190,642.532500,94

## Calculating source densities and false association radius (r_false) for each CSC source 

In [6]:
def annuli_area_deg2(r_in_arcsec, r_out_arcsec):
    r_in_deg = r_in_arcsec/3600
    r_out_deg = r_out_arcsec/3600
    area = np.pi*(r_out_deg**2 - r_in_deg**2)
    return area

def r_false(desi_rho_deg2, thresh = 0.03):
    ''' Belvedersky+ 2022 '''
    desi_rho_arcsec2 = desi_rho_deg2/(3600**2)
    return np.sqrt(-np.log(1-thresh)/(np.pi*desi_rho_arcsec2))


#make a dataframe of CSC sources
csc = pd.DataFrame(desi_csc.groupby(by = 'csc_name', ).agg({'csc_ra': np.mean, 'csc_dec': np.mean}))


#calculate total number of DESI sources withing 10-30 arcsec of each CSC source
num_src_desi_10_30 =  desi_csc.groupby(by = ['csc_name', pd.cut(desi_csc.dist_arcsec, [10, 30]) ]).size().unstack()


#calculate source densities for 10-30'' annulus
src_dens_deg2_10_30 = num_src_desi_10_30/annuli_area_deg2(10, 30)

#drop sources with  zeros in the 10-30'' source density (very few sources)
csc.drop(csc[src_dens_deg2_10_30 == 0].index, inplace=True)

#assign r_false for 10-30'' annulus density
csc['r_false_003_dens_10_30'] = r_false(src_dens_deg2_10_30)

csc.reset_index(inplace=True)

#assign r_false for each CSC source from csc dataframe to desi_csc dataframe
desi_csc = desi_csc.merge(csc[['csc_name','r_false_003_dens_10_30']], on='csc_name')

desi_csc.sample(10)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30
843423,2CXO J035112.6-274325,57.802726,-27.723746,NaN,1.379292,3.858086,9010,8336,176802,57.810348,-27.721426,1,0.085053,0.113917,0.292711,0.152291,-1.720966,96.070854,65.026300,3422.34180,2225.367000,230.885860,3.966890,0.920552,0.001482,0.000012,25.143604,24.836857,23.821777,24.541473,NaN,17.543280,17.967186,4.975658,5.373895,4.447730,0.303318,-1.651188,3.697979,0.223441,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,25.684846,9010_176802_8336,25.175776,25.175776,25.143604,24.858529,24.858529,24.836857,23.833902,23.833902,23.821777,24.543314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.543521,NaN,NaN,NaN,17.967277,NaN,NaN,NaN,4.941697e-16,0.306747,1.321827,1.015080,NaN,NaN,NaN,NaN,NaN,NaN,0.886591
2769574,2CXO J174717.4+451424,266.822548,45.240200,4.357568e-15,0.468630,1.310828,9011,2789,566100,266.815039,45.235346,1,2.121347,3.846339,5.115881,0.965885,0.471180,-24.592562,333.139530,674.49760,214.261230,106.932816,6.023781,1.846464,0.002177,0.000026,21.592377,20.976019,20.693375,22.532473,23.313830,NaN,16.193176,55.093708,56.301426,52.902473,2.370609,0.640262,-1.147491,1.691081,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,25.838510,9011_566100_2789,21.683471,21.683471,21.592377,21.037381,21.037381,20.976019,20.727699,20.727699,20.693375,22.537686,NaN,NaN,NaN,23.317033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.193435,NaN,NaN,NaN,8.815297e-15,0.616358,0.899002,0.282644,NaN,NaN,NaN,NaN,NaN,NaN,1.007624
3021598,2CXO J223745.6+183244,339.440039,18.545558,4.428502e-15,0.701760,1.962928,9010,5332,436515,339.445489,18.543132,1,0.576234,0.555003,1.011161,1.563645,2.005397,53.018050,-472.692660,171.71309,75.667854,9.093833,2.670909,0.613634,0.000736,0.000010,22.968367,23.051600,22.438915,22.007204,21.739923,18.187965,NaN,7.550928,4.827819,3.049255,2.555451,1.570924,1.438613,-1.521175,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,20.549161,9010_436515_5332,23.098503,23.098503,22.968367,23.139262,23.139262,23.051600,22.487949,NaN,NaN,22.014655,NaN,NaN,NaN,21.744499,NaN,NaN,NaN,18.188941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.083233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.007624
2817219,2CXO J195924.8-551603,299.853591,-55.267561,8.097683e-15,2.482928,6.945121,9010,3035,59672,299.850141,-55.260152,1,0.263411,0.541513,1.655260,1.572469,-2.230874,27.791811,-576.567500,636.34330,425.276370,32.430830,2.589238,0.685142,0.001047,0.000014,23.759869,23.038967,21.881792,21.997751,NaN,18.888794,NaN,6.644756,11.167214,9.426388,2.530277,-1.846568,0.899361,-2.147167,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,27.596523,9010_59672_3035,23.948415,23.948415,23.759869,23.165978,23.165978,23.038967,21.952834,21.952834,21.881792,22.008545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.890208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.950327e-15,0.720902,1.878077,1.157175,NaN,NaN,NaN,NaN,NaN,NaN,0.822722
1250834,2CXO J082219.6+704128,125.581730,70.691239,NaN,2.728682,7.632531,9011,2682,643348,125.595614,70.689842,1,1.101334,1.946946,1.042872,0.818627,1.208043,18.081654,730.464600,191.54270,76.541030,20.862179,3.838168,0.972741,0.000904,0.000012,22.313131,21.721330

In [7]:
#calculate for each CSC source the number of DESI sources within r_false
tmp_df = pd.DataFrame(desi_csc.query('dist_arcsec <= r_false_003_dens_10_30').groupby(by = 'csc_name').dist_arcsec.apply('count'))
print(tmp_df.dist_arcsec.value_counts())
tmp_df.columns = ['num_srcs_sep_less_r_false']

#merge with desi_csc dataframe
desi_csc = desi_csc.merge(tmp_df, on='csc_name')

1    72179
2     1853
3       41
4        1
Name: dist_arcsec, dtype: int64


## Counterpart  assignment

In [8]:
desi_csc_ctps = desi_csc.query('dist_arcsec <= r_false_003_dens_10_30 & num_srcs_sep_less_r_false==1 & dist_arcsec < csc_r_98 ')

In [9]:
print('delete ', desi_csc_ctps.duplicated(subset='desi_id').sum(), ' duplicated desi_id')
desi_csc_ctps = desi_csc_ctps.drop_duplicates(subset='desi_id')
print('Final number of counterparts:', len(desi_csc_ctps))
desi_csc_ctps['is_counterpart'] = True
#dublicates will appear here if untreated
#desi_csc_ctps.pivot_table(values='name_csc', index='desi_id', aggfunc='count').sort_values(by = 'name_csc',ascending=False)

delete  1  duplicated desi_id
Final number of counterparts: 72021


## Field source assignment

In [10]:
desi_csc_field = desi_csc.query("dist_arcsec>=10 & dist_arcsec<=50 ")
print('number of possible field sources: ', len(desi_csc_field))

number of possible field sources:  1410131


In [17]:
tmp_df = desi_csc_field.groupby(by = 'desi_id', ).agg({'csc_name': 'count'})
clear_field_ids = tmp_df.index[tmp_df['csc_name']==1]
desi_csc_field = desi_csc_field[desi_csc_field.desi_id.isin(clear_field_ids)==True]
fraction_to_retain = 0.3
desi_csc_field = desi_csc_field.sample(frac=fraction_to_retain)
desi_csc_field['is_counterpart'] = False
desi_csc_field['csc_flux_05_2'] = np.nan
desi_csc_field['csc_r_98'] = np.nan

print(f'Final number of secure field sources  ({fraction_to_retain} of it): ', len(desi_csc_field))

Final number of secure field sources  (0.3 of it):  368740


## Saving the combined catalogs

In [18]:
desi_csc_training = pd.concat([desi_csc_ctps, desi_csc_field])
#save cat to pickle
desi_csc_training.to_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')